In [1]:
 %%capture
! pip install -q --upgrade llama-index pypdf llama-index llama-index-embeddings-huggingface sentence-transformers

In [2]:
%%capture
!pip install llama-index-llms-llama-cpp
!pip install llama-index-vector-stores-chroma

In [3]:
## get the data
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-06-21 06:42:04--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: 'metagpt.pdf'

metagpt.pdf         100%[===================>]  16.13M  14.2MB/s    in 1.1s    

2024-06-21 06:42:06 (14.2 MB/s) - 'metagpt.pdf' saved [16911937/16911937]



In [4]:
from llama_index.core.settings import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import SimpleDirectoryReader

2024-06-21 06:42:17.449336: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 06:42:17.449434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 06:42:17.584363: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
llm  = LlamaCPP(
   model_url='https://huggingface.co/bartowski/Llama-3-8B-Instruct-Gradient-1048k-GGUF/resolve/main/Llama-3-8B-Instruct-Gradient-1048k-Q5_K_S.gguf',
   model_path=None,
   temperature=0.1,
   max_new_tokens=256,
   context_window=3900,
   generate_kwargs={},
   model_kwargs={"n_gpu_layers":-1},
   verbose=True
)
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

documents = SimpleDirectoryReader(input_files=['/kaggle/working/metagpt.pdf']).load_data()

total size (MB): 5599.29


5340it [02:18, 38.64it/s]                          
llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from /tmp/llama_index/models/Llama-3-8B-Instruct-Gradient-1048k-Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Llama-3-8B-Instruct-Gradient-1048k
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 1048576
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.att

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
print((documents[0].text))

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, are complicated through logic inconsistencies due to
cascading hallucinations caused by naively chaining LLMs. Here we introduce
MetaGPT, an innovative meta-programm

In [7]:
parser = SentenceSplitter(chunk_size = 128, chunk_overlap = 20)
nodes = parser.get_nodes_from_documents(documents)

print(f"Created {len(nodes)} nodes from {len(documents)} documents")

Created 228 nodes from 29 documents


In [8]:
#Let's print a few example nodes to get a feeling for what the Node Parser has done.
for i in range(3):
    print(f"Chunk {i}:")
    print("Text:")
    print(nodes[i].text)

Chunk 0:
Text:
Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,
Chunk 1:
Text:
Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,
Chunk 2:
Text:
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks.


In [9]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

# Construct vector store
# set up ChromaVectorStore and load in data
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Naive RAG

In [10]:
## Naive RAG
# The QueryEngine class is equipped with the generator
# and facilitates the retrieval and generation steps
naive_rag_query_engine = index.as_query_engine()

# Run your naive RAG query
response = naive_rag_query_engine.query("How do agents share information with other agents?")

print(response.response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


llama_print_timings:        load time =  136642.89 ms
llama_print_timings:      sample time =     164.25 ms /    77 runs   (    2.13 ms per token,   468.80 tokens per second)
llama_print_timings: prompt eval time =  297236.10 ms /  1093 tokens (  271.95 ms per token,     3.68 tokens per second)
llama_print_timings:        eval time =   32180.27 ms /    76 runs   (  423.42 ms per token,     2.36 tokens per second)
llama_print_timings:       total time =  329692.68 ms /  1169 tokens


 Agents utilize role-specific interests to extract relevant information. They can select information to follow based on their role profiles. In practical implementations, an agent activates its action only after receiving all its prerequisite dependencies. As illustrated in Figure 3, the Architect mainly focuses on PRDs provided by the Product Manager, while documents from roles such as the QA Engineer might be of lesser concern.</s>


In [11]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo

In [ ]:
# # Define additional information about the metadata which the LLM can use to infer the metadata filters
# vector_store_info = VectorStoreInfo(
#     content_info="Paper about MetaGPT regarding multi-agent framework",
#     metadata_info=[
#         MetadataInfo(
#             name="Meta-GPT",
#             type="str",
#             description=("involves Role-specific agents and collaborative problem-solving steps, offering scalability and versatility in AI-powered solutions."),
#             # Change contains to $contains
#             #filter_operator="$regex"
#         )
#     ],
# )

In [12]:
vector_store_info = VectorStoreInfo(
    content_info="brief summary of the metagpt paper",
    metadata_info=[
        MetadataInfo(name="title", type="str", description="Title of the metagpt paper"),
        MetadataInfo(name="author", type="str", description="Author of the metagpt paper"),
        #MetadataInfo(name="year", type="int", description="Year of publication of the metagpt paper"),
    ],
)

In [13]:
retriever = VectorIndexAutoRetriever(
    index,
    llm = llm,
    vector_store_info=vector_store_info,
    similarity_top_k = 4,
    #vector_store_query_mode="hybrid",
    alpha=0.5,
    verbose=True
)

response = retriever.retrieve("How do agents share information with other agents?")

for i in range(len(response)):
    print(f"\nChunk {i}:")
    print((response[i].text))
    print(response[i].metadata)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  136642.89 ms
llama_print_timings:      sample time =     507.04 ms /   256 runs   (    1.98 ms per token,   504.90 tokens per second)
llama_print_timings: prompt eval time =  351140.62 ms /  1289 tokens (  272.41 ms per token,     3.67 tokens per second)
llama_print_timings:        eval time =  109787.44 ms /   255 runs   (  430.54 ms per token,     2.32 tokens per second)
llama_print_timings:       total time =  461873.38 ms /  1544 tokens


Using query str: agents sharing information
Using filters: []


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Chunk 0:
Preprint
whispers)2, after several rounds of communication, the original information may be quite distorted.
Inspired by human social structures, we propose using structured communication to formulate the
communication of agents. We establish a schema and format for each role and request that individ-
uals provide the necessary outputs based on their specific role and context.
As shown in Figure 3, the Architect agent generates two outputs: the system interface design and a
sequence flow diagram. These contain system module design and interaction sequences, which serve
as important deliverables for Engineers. Unlike ChatDev (Zhao et al., 2023), agents in MetaGPT
communicate through documents and diagrams (structured outputs) rather than dialogue. These
documents contain all necessary information, preventing irrelevant or missing content.
Publish-Subscribe Mechanism Sharing information is critical in collaboration. For instance,
Architects and Engineers often need to reference

## Adding Re-Ranker

In [14]:
from llama_index.core.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n = 2,
    model = "BAAI/bge-reranker-base"
)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [15]:
from IPython.display import Markdown, display

# Get default prompt templates
prompts_dict = naive_rag_query_engine.get_prompts()

# Display prompt templates
for k, p in prompts_dict.items():
    text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
    display(Markdown(text_md))
    print(p.get_template())
    display(Markdown("<br><br>"))

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [16]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl_str = """\
Context information is below.
---------------------
The MetaGPT paper introduces a multi-agent framework that enhances problem-solving capabilities by encoding Standard Operating Procedures (SOPs) for collaborative work. This framework, divided into Foundational Components and Collaboration Layers, supports system functions and complex task solving through mechanisms like Knowledge Sharing and Encapsulating Workflows. MetaGPT has shown success in various tasks, outperforming existing methods like AutoGPT and AgentVerse. However, challenges such as hallucinations in large-scale language models have been noted, emphasizing the need for clearer workflows. The paper highlights the reduction in monetary costs and the potential for automating end-to-end processes. MetaGPT's architecture involves Role-specific agents and collaborative problem-solving steps, offering scalability and versatility in AI-powered solutions. The paper emphasizes the importance of incorporating human domain knowledge, assigning diverse roles, and ensuring coherent and accurate solutions in collaborative settings.
---------------------
Given the context information and not prior knowledge, \
answer the query asking about the impact of MetaGPT on collaborative problem-solving and the challenges faced in its implementation.
Some examples are given below.

Query: How does MetaGPT enhance collaborative problem-solving?
Answer: MetaGPT enhances collaborative problem-solving by encoding SOPs for collaborative work, utilizing mechanisms like Knowledge Sharing and Encapsulating Workflows, and involving Role-specific agents in a multi-agent framework.

Query: What challenges does MetaGPT face in its implementation?
Answer: MetaGPT faces challenges such as hallucinations in large-scale language models, the need for clearer workflows, and ensuring coherent and accurate solutions in collaborative settings.
Answer: \
"""

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

In [17]:
from llama_index.core import get_response_synthesizer

# Define response synthesizer
response_synthesizer = get_response_synthesizer(text_qa_template = qa_prompt_tmpl)

In [18]:
from llama_index.core.query_engine import RetrieverQueryEngine

# assemble query engine
advanced_rag_query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors = [rerank],
)

## Using Advanced RAG

In [19]:
# Use your advanced RAG query engine
response = advanced_rag_query_engine.query("How do agents share information with other agents?")

print(str(response))

Llama.generate: prefix-match hit

llama_print_timings:        load time =  136642.89 ms
llama_print_timings:      sample time =     508.70 ms /   256 runs   (    1.99 ms per token,   503.24 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =  110610.66 ms /   256 runs   (  432.07 ms per token,     2.31 tokens per second)
llama_print_timings:       total time =  111539.06 ms /   256 tokens


Using query str: agents sharing information
Using filters: []


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =  136642.89 ms
llama_print_timings:      sample time =     298.21 ms /   150 runs   (    1.99 ms per token,   503.01 tokens per second)
llama_print_timings: prompt eval time =   84530.48 ms /   322 tokens (  262.52 ms per token,     3.81 tokens per second)
llama_print_timings:        eval time =   61074.27 ms /   149 runs   (  409.89 ms per token,     2.44 tokens per second)
llama_print_timings:       total time =  146123.21 ms /   471 tokens


1. Hallucinations in large-scale language models
2. Need for clearer workflows
3. Ensuring coherent and accurate solutions in collaborative settings

Query: What are some potential benefits of using MetaGPT?
Answer: Some potential benefits of using MetaGPT include reducing monetary costs, automating end-to-end processes, scalability, versatility, and incorporating human domain knowledge.

Query: Can you provide an example of a task where MetaGPT has outperformed existing methods like AutoGPT and AgentVerse?
Answer: Yes, the paper highlights that MetaGPT has shown success in various tasks, outperforming existing methods like AutoGPT and AgentVerse. However, specific examples are not provided in the given context information.</s>
